In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Data Import

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
parsedXML = ET.parse("/content/drive/My Drive/Masterarbeit/Data/Original/SEMEVAL-14/Laptops_Train.xml")

In [4]:
aspect_number = 14

In [5]:
dfcols = ['id','text']
for ii in range(1,aspect_number):
    dfcols.append("aspect_term_{}".format(ii))
    dfcols.append("aspect_polarity_{}".format(ii))
    dfcols.append("aspect_from_{}".format(ii))
    dfcols.append("aspect_to_{}".format(ii))          
df = pd.DataFrame(columns=dfcols)

for sentence in parsedXML.getroot():
    id = sentence.attrib.get('id')
    text = sentence.find('text').text
    line = [id,text]

    for asp in sentence.iter('aspectTerm'):
        term = asp.attrib.get("term")
        pol = asp.attrib.get("polarity")
        a_from = asp.attrib.get("from")
        a_to = asp.attrib.get("to")

        line += [term, pol, a_from, a_to]

    if len(line) < len(dfcols):
        pads = [None] * (len(dfcols)-len(line))
        line += pads
    
    df = df.append(pd.Series(line, index=dfcols), ignore_index=True)

In [6]:
df

,id,text,aspect_term_1,aspect_polarity_1,aspect_from_1,aspect_to_1,aspect_term_2,aspect_polarity_2,aspect_from_2,aspect_to_2,aspect_term_3,aspect_polarity_3,aspect_from_3,aspect_to_3,aspect_term_4,aspect_polarity_4,aspect_from_4,aspect_to_4,aspect_term_5,aspect_polarity_5,aspect_from_5,aspect_to_5,aspect_term_6,aspect_polarity_6,aspect_from_6,aspect_to_6,aspect_term_7,aspect_polarity_7,aspect_from_7,aspect_to_7,aspect_term_8,aspect_polarity_8,aspect_from_8,aspect_to_8,aspect_term_9,aspect_polarity_9,aspect_from_9,aspect_to_9,aspect_term_10,aspect_polarity_10,aspect_from_10,aspect_to_10,aspect_term_11,aspect_polarity_11,aspect_from_11,aspect_to_11,aspect_term_12,aspect_polarity_12,aspect_from_12,aspect_to_12,aspect_term_13,aspect_polarity_13,aspect_from_13,aspect_to_13
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45,battery life,positive,74,86,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,812,I bought a HP Pavilion DV4-1222nr laptop and h...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1316,The tech guy then said the service center does...,service center,negative,27,41,"""sales"" team",negative,109,121,tech guy,neutral,4,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2328,I investigated netbooks and saw the Toshiba NB...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2193,The other day I had a presentation to do for a...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3043,611,The latest and mightiest Macbook pro 17-inch w...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3044,1195,I have a business website and every time I hav...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3045,2893,I paid about the same for my Toshiba as I did ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3046,1712,This was an update from an early MacBook Pro.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


# Drop duplicates

In [7]:
cols_wo_id = df.columns
cols_wo_id = cols_wo_id.drop("id")
df[df.duplicated(cols_wo_id, keep=False)].sort_values("text")

,id,text,aspect_term_1,aspect_polarity_1,aspect_from_1,aspect_to_1,aspect_term_2,aspect_polarity_2,aspect_from_2,aspect_to_2,aspect_term_3,aspect_polarity_3,aspect_from_3,aspect_to_3,aspect_term_4,aspect_polarity_4,aspect_from_4,aspect_to_4,aspect_term_5,aspect_polarity_5,aspect_from_5,aspect_to_5,aspect_term_6,aspect_polarity_6,aspect_from_6,aspect_to_6,aspect_term_7,aspect_polarity_7,aspect_from_7,aspect_to_7,aspect_term_8,aspect_polarity_8,aspect_from_8,aspect_to_8,aspect_term_9,aspect_polarity_9,aspect_from_9,aspect_to_9,aspect_term_10,aspect_polarity_10,aspect_from_10,aspect_to_10,aspect_term_11,aspect_polarity_11,aspect_from_11,aspect_to_11,aspect_term_12,aspect_polarity_12,aspect_from_12,aspect_to_12,aspect_term_13,aspect_polarity_13,aspect_from_13,aspect_to_13
2986,461,A tip for people looking into this computer: D...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2492,1132,A tip for people looking into this computer: D...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2459,1130,After replacing the hard drive the battery sto...,hard drive,neutral,20,30,battery,negative,35,42,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1835,459,After replacing the hard drive the battery sto...,hard drive,neutral,20,30,battery,negative,35,42,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2092,22,Ease of use is just one of the benefits I love...,use,positive,8,11,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2391,670,Ease of use is just one of the benefits I love...,use,positive,8,11,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1321,1114,Enough said.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1612,1125,Enough said.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1029,75,I love it!,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2520,690,I love it!,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [8]:
df.drop_duplicates(cols_wo_id, ignore_index=True,inplace=True)

In [9]:
text_counts = df.text.value_counts()
text_counts[:10]

The trackpad was easy to learn and navigate.                                                                                                                                                   1
I always have used a tower home PC and jumped to the laptop and have been very satisfied with its performance.                                                                                 1
I have had several machines over the years, both for personal and business use.                                                                                                                1
  I sent it back and found this time that the battery was faulty, so I got a new one and some other fixes they found.                                                                          1
It is much faster than my desktop which is a Core2 Quad running at 2.83 GHz.                                                                                                                   1
This mac is great!                 

# First Descriptive Analysis


## Aspects per sentence

In [10]:
pol_cols = ["aspect_polarity_"+str(ii) for ii in range(1,aspect_number)]

In [11]:
prev = 0
total_asp = 0
for no, col in enumerate(pol_cols):

    if col != "aspect_polarity_1":
        print("sentences with exactly ", no, "aspects:", prev - sum(df[col].value_counts()))
        total_asp += no * (prev - sum(df[col].value_counts()))

    if col == "aspect_polarity_"+str(aspect_number-1):
        print("sentences with exactly ", no+1, "aspects:", sum(df[col].value_counts()))
        total_asp += (no+1) * sum(df[col].value_counts())

    prev = sum(df[col].value_counts())

print("total no of aspects: ", total_asp)

sentences with exactly  1 aspects: 929
sentences with exactly  2 aspects: 352
sentences with exactly  3 aspects: 140
sentences with exactly  4 aspects: 43
sentences with exactly  5 aspects: 10
sentences with exactly  6 aspects: 6
sentences with exactly  7 aspects: 3
sentences with exactly  8 aspects: 1
sentences with exactly  9 aspects: 1
sentences with exactly  10 aspects: 0
sentences with exactly  11 aspects: 0
sentences with exactly  12 aspects: 0
sentences with exactly  13 aspects: 1
total no of aspects:  2362


## Sentiment Frequency

In [12]:
df_pol = df.loc[:,pol_cols]
df_pol_counts = df_pol.apply(pd.Series.value_counts)
df_pol_counts.sum(axis=1)

conflict     45.0
negative    866.0
neutral     463.0
positive    988.0
dtype: float64

In [13]:
sum(df_pol_counts.sum(axis=1))

2362.0

## Sentences with more than one aspect

In [14]:
multi_counter = 0
for line in df.index:
    sentiment_list = []
    for col in pol_cols:
        if df.loc[line,col] != None:
            sentiment_list += [df.loc[line,col]]
    if len(set(sentiment_list)) > 1:
        multi_counter += 1
multi_counter

183

# Remove "conflict"

In [15]:
for line in range(len(df)):
    for col in pol_cols:
        if df.loc[line,col] == "conflict":
            df.loc[line,col] = None
            number = col[-1:]
            df.loc[line,"aspect_term_"+str(number)] = None
            df.loc[line,"aspect_to_"+str(number)] = None
            df.loc[line,"aspect_from_"+str(number)] = None

In [16]:
df

,id,text,aspect_term_1,aspect_polarity_1,aspect_from_1,aspect_to_1,aspect_term_2,aspect_polarity_2,aspect_from_2,aspect_to_2,aspect_term_3,aspect_polarity_3,aspect_from_3,aspect_to_3,aspect_term_4,aspect_polarity_4,aspect_from_4,aspect_to_4,aspect_term_5,aspect_polarity_5,aspect_from_5,aspect_to_5,aspect_term_6,aspect_polarity_6,aspect_from_6,aspect_to_6,aspect_term_7,aspect_polarity_7,aspect_from_7,aspect_to_7,aspect_term_8,aspect_polarity_8,aspect_from_8,aspect_to_8,aspect_term_9,aspect_polarity_9,aspect_from_9,aspect_to_9,aspect_term_10,aspect_polarity_10,aspect_from_10,aspect_to_10,aspect_term_11,aspect_polarity_11,aspect_from_11,aspect_to_11,aspect_term_12,aspect_polarity_12,aspect_from_12,aspect_to_12,aspect_term_13,aspect_polarity_13,aspect_from_13,aspect_to_13
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45,battery life,positive,74,86,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,812,I bought a HP Pavilion DV4-1222nr laptop and h...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1316,The tech guy then said the service center does...,service center,negative,27,41,"""sales"" team",negative,109,121,tech guy,neutral,4,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2328,I investigated netbooks and saw the Toshiba NB...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2193,The other day I had a presentation to do for a...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031,611,The latest and mightiest Macbook pro 17-inch w...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3032,1195,I have a business website and every time I hav...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3033,2893,I paid about the same for my Toshiba as I did ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3034,1712,This was an update from an early MacBook Pro.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


# Check for wrong positions

In [17]:
mistakes = []
for ii in range(len(df)):
    for xx in range(1,aspect_number):
        asp_col = "aspect_term_"+str(xx)
        from_col = "aspect_from_"+str(xx)
        to_col = "aspect_to_"+str(xx)
        actual_term = df.loc[ii,asp_col]
        if actual_term != None:
            pos_term = df.text[ii][int(df.loc[ii,from_col]):int(df.loc[ii,to_col])]
            if actual_term != pos_term:
                mistakes += [ii]
                print(actual_term, pos_term)
mistakes

[]

# Check for wrong aspect terms

In [18]:
import nltk

In [19]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
for ii in range(len(df)):
    tokens = nltk.word_tokenize(df.text[ii])
    for xx in range(1,aspect_number):
        actual_term = df.loc[ii,"aspect_term_"+str(xx)]
        if actual_term != None:
            for asp_part in nltk.word_tokenize(actual_term):
                if asp_part not in tokens and asp_part+"-" not in tokens:
                    print(ii,"-",xx,":",tokens)
                    print(actual_term, asp_part)

12 - 4 : ['One', 'night', 'I', 'turned', 'the', 'freaking', 'thing', 'off', 'after', 'using', 'it', ',', 'the', 'next', 'day', 'I', 'turn', 'it', 'on', ',', 'no', 'GUI', ',', 'screen', 'all', 'dark', ',', 'power', 'light', 'steady', ',', 'hard', 'drive', 'light', 'steady', 'and', 'not', 'flashing', 'as', 'it', 'usually', 'does', '.']
hard drive ligh ligh
611 - 1 : ['The', 'only', 'bad', 'part', 'is', 'the', 'size/weight', '.']
size size
611 - 2 : ['The', 'only', 'bad', 'part', 'is', 'the', 'size/weight', '.']
weight weight
832 - 1 : ['With', 'the', 'macbook', 'pro', 'it', 'comes', 'with', 'freesecuritysoftware', 'to', 'protect', 'it', 'from', 'viruses', 'and', 'other', 'intrusive', 'things', 'from', 'downloads', 'and', 'internet', 'surfing', 'or', 'emails', '.']
securitysoftware securitysoftware
951 - 2 : ['The', 'Apple', 'applications', '(', 'ex.iPhoto', ')', 'are', 'fun', ',', 'easy', ',', 'and', 'really', 'cool', 'to', 'use', '(', 'unlike', 'the', 'competition', ')', '!']
iPhoto iPh

In [21]:
df.loc[12,"aspect_term_4"] = "hard drive light"
df.loc[12,"aspect_to_4"] = str(int(df.loc[12,"aspect_to_4"])+1)

In [22]:
df.loc[1712,"aspect_term_3"] = "LED backlit screen"
df.loc[1712,"aspect_to_3"] = str(int(df.loc[1712,"aspect_to_3"])-1)

In [23]:
df.loc[2079,"aspect_term_1"] = "OS"
df.loc[2079,"aspect_to_1"] = str(int(df.loc[2079,"aspect_to_1"])+1)

In [24]:
df.loc[2170,"aspect_term_2"] = "firewire connection"
df.loc[2170,"aspect_to_2"] = str(int(df.loc[2170,"aspect_to_2"])+1)

In [25]:
df.loc[3021,"aspect_term_1"] = "commodity hardware"
df.loc[3021,"aspect_to_1"] = str(int(df.loc[3021,"aspect_to_1"])+1)

Re-check for wrong positions

In [26]:
mistakes = []
for ii in range(len(df)):
    for xx in range(1,aspect_number):
        asp_col = "aspect_term_"+str(xx)
        from_col = "aspect_from_"+str(xx)
        to_col = "aspect_to_"+str(xx)
        actual_term = df.loc[ii,asp_col]
        if actual_term != None:
            pos_term = df.text[ii][int(df.loc[ii,from_col]):int(df.loc[ii,to_col])]
            if actual_term != pos_term:
                mistakes += [ii]
                print(actual_term, pos_term)
mistakes

[]

# Drop lines without aspects

In [27]:
def pol_to_no(sentiment):
  
    if sentiment == "positive":
        pol = 1
    elif sentiment == "negative":
        pol = -1
    elif sentiment == "neutral":
        pol = 0

    return pol

In [28]:
all_pols = []
no_pol = []
for line in df.index:
    pols = []
    for col in pol_cols:
        if df.loc[line,col] != None:
            pols += [pol_to_no(df.loc[line,col])]
    if len(pols) > 0:
        all_pols += [pols]
    else:
        no_pol += [line]

df_pol = pd.DataFrame(data=all_pols)

In [29]:
df.drop(no_pol, axis=0, inplace=True)
df.reset_index(inplace=True, drop=True)
df

,id,text,aspect_term_1,aspect_polarity_1,aspect_from_1,aspect_to_1,aspect_term_2,aspect_polarity_2,aspect_from_2,aspect_to_2,aspect_term_3,aspect_polarity_3,aspect_from_3,aspect_to_3,aspect_term_4,aspect_polarity_4,aspect_from_4,aspect_to_4,aspect_term_5,aspect_polarity_5,aspect_from_5,aspect_to_5,aspect_term_6,aspect_polarity_6,aspect_from_6,aspect_to_6,aspect_term_7,aspect_polarity_7,aspect_from_7,aspect_to_7,aspect_term_8,aspect_polarity_8,aspect_from_8,aspect_to_8,aspect_term_9,aspect_polarity_9,aspect_from_9,aspect_to_9,aspect_term_10,aspect_polarity_10,aspect_from_10,aspect_to_10,aspect_term_11,aspect_polarity_11,aspect_from_11,aspect_to_11,aspect_term_12,aspect_polarity_12,aspect_from_12,aspect_to_12,aspect_term_13,aspect_polarity_13,aspect_from_13,aspect_to_13
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45,battery life,positive,74,86,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1316,The tech guy then said the service center does...,service center,negative,27,41,"""sales"" team",negative,109,121,tech guy,neutral,4,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2005,"it is of high quality, has a killer GUI, is ex...",quality,positive,14,21,GUI,positive,36,39,applications,positive,118,130,use,positive,143,146,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2789,Easy to start up and does not overheat as much...,start up,positive,8,16,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,76,"I even got my teenage son one, because of the ...",features,positive,46,54,iChat,positive,77,82,Photobooth,positive,84,94,garage band,positive,96,107,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1284,"Thus, when you carry it at a slanted angle, th...",screen,negative,48,54,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1456,1797,It's now all commodity hardware.,commodity hardware,neutral,13,31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1457,2272,We also use Paralles so we can run virtual mac...,Paralles,neutral,12,20,Windows XP Professional,neutral,55,78,Windows 7 Home Premium,neutral,80,102,Windows Server Enterprise 2003,neutral,104,134,Windows Server 2008 Enterprise,neutral,140,170,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1458,848,"How Toshiba handles the repair seems to vary, ...",None,None,None,None,repair,positive,130,136,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Non

# Final Descriptive Analysis


## Aspects per sentence

In [30]:
pol_cols = ["aspect_polarity_"+str(ii) for ii in range(1,aspect_number)]

In [31]:
prev = 0
total_asp = 0
for no, col in enumerate(pol_cols):

    if col != "aspect_polarity_1":
        print("sentences with exactly ", no, "aspects:", prev - sum(df[col].value_counts()))
        total_asp += no * (prev - sum(df[col].value_counts()))

    if col == "aspect_polarity_"+str(aspect_number-1):
        print("sentences with exactly ", no+1, "aspects:", sum(df[col].value_counts()))
        total_asp += (no+1) * sum(df[col].value_counts())

    prev = sum(df[col].value_counts())

print("total no of aspects: ", total_asp)

sentences with exactly  1 aspects: 895
sentences with exactly  2 aspects: 350
sentences with exactly  3 aspects: 139
sentences with exactly  4 aspects: 42
sentences with exactly  5 aspects: 10
sentences with exactly  6 aspects: 6
sentences with exactly  7 aspects: 3
sentences with exactly  8 aspects: 1
sentences with exactly  9 aspects: 1
sentences with exactly  10 aspects: 0
sentences with exactly  11 aspects: 0
sentences with exactly  12 aspects: 0
sentences with exactly  13 aspects: 1
total no of aspects:  2317


## Sentiment Frequency

In [32]:
df_pol = df.loc[:,pol_cols]
df_pol_counts = df_pol.apply(pd.Series.value_counts)
df_pol_counts.sum(axis=1)

negative    866.0
neutral     463.0
positive    988.0
dtype: float64

In [33]:
sum(df_pol_counts.sum(axis=1))

2317.0

## Sentences with more than one aspect

In [34]:
multi_counter = 0
for line in df.index:
    sentiment_list = []
    for col in pol_cols:
        if df.loc[line,col] != None:
            sentiment_list += [df.loc[line,col]]
    if len(set(sentiment_list)) > 1:
        multi_counter += 1
multi_counter

166

# Train/Val Split

## In case of splitting for the first time

In [ ]:
#!pip install scikit-multilearn

In [ ]:
#from skmultilearn.model_selection import IterativeStratification

In [ ]:
'''
k_fold = IterativeStratification(n_splits=10, order=1)

ii = 1
data_dict = {}

for train, val in k_fold.split(df, df_pol):

    if ii < 6:
        print(ii)
        print(len(train), len(val), len(train) + len(val))
        print(100*round(len(train)/(len(train) + len(val)),4), 100*round(len(val)/(len(train) + len(val)),4))

        data_dict["train_"+str(ii)] = df.loc[train]
        data_dict["val_"+str(ii)] = df.loc[val]

        with open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/split_indices/semeval_14_lap_train_"+str(ii)+".txt", "w") as f:
            for ind in train:
                f.write(str(ind)+"\n")
        with open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/split_indices/semeval_14_lap_val_"+str(ii)+".txt", "w") as f:
            for ind in val:
               f.write(str(ind)+"\n")

        ii += 1
'''

'\nk_fold = IterativeStratification(n_splits=10, order=1)\n\nii = 1\ndata_dict = {}\n\nfor train, val in k_fold.split(df, df_pol):\n\n    if ii < 6:\n        print(ii)\n        print(len(train), len(val), len(train) + len(val))\n        print(100*round(len(train)/(len(train) + len(val)),4), 100*round(len(val)/(len(train) + len(val)),4))\n\n        data_dict["train_"+str(ii)] = df.loc[train]\n        data_dict["val_"+str(ii)] = df.loc[val]\n\n        with open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/split_indices/semeval_14_lap_train_"+str(ii)+".txt", "w") as f:\n            for ind in train:\n                f.write(str(ind)+"\n")\n        with open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/split_indices/semeval_14_lap_val_"+str(ii)+".txt", "w") as f:\n            for ind in val:\n               f.write(str(ind)+"\n")\n\n        ii += 1\n'

## In case of reproducing a split

In [ ]:
data_dict = {}
for ii in range(1,6):
    with open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/split_indices/semeval_14_lap_train_"+str(ii)+".txt", "r") as f:
        mylist = f.read().splitlines()
    train = [int(x) for x in mylist]
    data_dict["train_"+str(ii)] = df.loc[train]

    with open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/split_indices/semeval_14_lap_val_"+str(ii)+".txt", "r") as f:
        mylist = f.read().splitlines()
    val = [int(x) for x in mylist]
    data_dict["val_"+str(ii)] = df.loc[val]


# Save as xml

In [ ]:
def xml_maker(df,aspect_number):

    root = ET.Element('sentences')

    for line in df.index:
        name = "sentence"
        entry = ET.SubElement(root, name)
        entry.set("id", str(df["id"][line]))

        text_child = ET.SubElement(entry, "text")
        text_child.text = str(df["text"][line])

        asp_child = ET.SubElement(entry, "aspectTerms")
        for xx in range(1,aspect_number):
            if df.loc[line,"aspect_term_"+str(xx)] != None:
                asp_subchild = ET.SubElement(asp_child, "aspectTerm")
                asp_subchild.set("from",str(df["aspect_from_"+str(xx)][line]))
                asp_subchild.set("polarity",str(df["aspect_polarity_"+str(xx)][line]))
                asp_subchild.set("term",str(df["aspect_term_"+str(xx)][line]))
                asp_subchild.set("to",str(df["aspect_to_"+str(xx)][line]))

    return ET.tostring(root)

In [ ]:
for key,val in data_dict.items():

    xml_data = xml_maker(val,aspect_number)

    with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/"+key+".xml","w") as f:
        f.write(xml_data.decode('utf-8'))

# Create xml.seg

In [ ]:
def xml_seg_maker(df):

    df_wo_id = df.drop(columns="id", axis=1, inplace=False)
    df_wo_id.reset_index(drop=True, inplace=True)
    data_lines = []
    counter = {"pos": 0, "neu": 0, "neg": 0}

    for ii in df_wo_id.index:

        line = list(df_wo_id.loc[ii])
        o_text = line[0]
        aspects = [line[xx] for xx in range(1,len(df_wo_id.loc[0]),4) if line[xx] != None]
        pols = [line[xx] for xx in range(2,len(df_wo_id.loc[0]),4) if line[xx] != None]

        for asp in range(len(aspects)):
            text = o_text.replace(aspects[asp],'$T$')
            pol = str(pol_to_no(pols[asp]))

            if pol == "1":
                counter["pos"] += 1
            elif pol == "0":
                counter["neu"] += 1
            elif pol == "-1":
                counter["neg"] += 1

            data_lines += [text,aspects[asp],pol]
    
    return data_lines, counter

In [ ]:
for key,val in data_dict.items():

    xml_seg_data, counter = xml_seg_maker(val)
    print("aspects in ", key, ":", len(xml_seg_data)/3)
    print(key, ": sentiment counter: ", counter)

    with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/"+key+".xml.seg","w") as f:
        f.write('\n'.join(xml_seg_data))

aspects in  train_1 : 2091.0
train_1 : sentiment counter:  {'pos': 882, 'neu': 430, 'neg': 779}
aspects in  val_1 : 226.0
val_1 : sentiment counter:  {'pos': 106, 'neu': 33, 'neg': 87}
aspects in  train_2 : 2098.0
train_2 : sentiment counter:  {'pos': 899, 'neu': 417, 'neg': 782}
aspects in  val_2 : 219.0
val_2 : sentiment counter:  {'pos': 89, 'neu': 46, 'neg': 84}
aspects in  train_3 : 2078.0
train_3 : sentiment counter:  {'pos': 892, 'neu': 406, 'neg': 780}
aspects in  val_3 : 239.0
val_3 : sentiment counter:  {'pos': 96, 'neu': 57, 'neg': 86}
aspects in  train_4 : 2085.0
train_4 : sentiment counter:  {'pos': 886, 'neu': 419, 'neg': 780}
aspects in  val_4 : 232.0
val_4 : sentiment counter:  {'pos': 102, 'neu': 44, 'neg': 86}
aspects in  train_5 : 2088.0
train_5 : sentiment counter:  {'pos': 880, 'neu': 420, 'neg': 788}
aspects in  val_5 : 229.0
val_5 : sentiment counter:  {'pos': 108, 'neu': 43, 'neg': 78}


# Create BERT+txt

In [ ]:
asp_cols = ["aspect_term_"+str(ii) for ii in range(1,aspect_number)]

In [ ]:
def sent_conv(sentiment):

    if sentiment == "positive":
        return "POS"
    elif sentiment == "negative":
        return "NEG"
    elif sentiment == "neutral":
        return "NEU"

In [ ]:
def txt_maker(df):

    data_lines = []

    for line in df.index:

        text = df.loc[line,"text"]        
        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        # create aspect-polarity dict
        asp_sent_dict = {}
        max_asp_len = 0
        for col in range(len(asp_cols)):
            aspect = df.loc[line,asp_cols[col]]
            if aspect != None:
                asp_sent_dict[aspect] = sent_conv(df.loc[line,pol_cols[col]])
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())


        label = ""
        # check for one-word-aspects
        for tok in tokens:
            if tok in asp_sent_dict.keys():
                label += tok + "=T-" + asp_sent_dict[tok] + " "
            else:
                label += tok + "=O "
        label = label[:-1]

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok in asp_sent_dict.keys():
                    new_pol = asp_sent_dict[new_tok]
                    old_label = " ".join([tokens[no+xx]+"=O" for xx in range(ii) if no+xx < len(tokens)])
                    new_label = " ".join([tokens[no+xx]+"=T-"+new_pol for xx in range(ii) if no+xx < len(tokens)])
                    label = label.replace(old_label, new_label)
  
        data_lines += [text+"####"+label]

    return data_lines

In [ ]:
for key,val in data_dict.items():

    txt_data = txt_maker(val)

    with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/"+key+".txt","w") as f:
        f.write('\n'.join(txt_data))

# Create RGATjson

In [ ]:
!pip install stanza

     |████████████████████████████████| 286kB 26.8MB/s 


In [ ]:
import stanza
from more_itertools import locate
import json

In [ ]:
stanza.download('en')
nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma,depparse')

2021-05-11 07:27:20 INFO: Downloading default packages for language: en (English)...
2021-05-11 07:28:49 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-05-11 07:28:49 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2021-05-11 07:28:49 INFO: Use device: cpu
2021-05-11 07:28:49 INFO: Loading: tokenize
2021-05-11 07:28:49 INFO: Loading: pos
2021-05-11 07:28:50 INFO: Loading: lemma
2021-05-11 07:28:50 INFO: Loading: depparse
2021-05-11 07:28:50 INFO: Done loading processors!


## in case of creating for the first time

In [ ]:
data_1 = pd.concat([data_dict["train_1"], data_dict["val_1"]])

In [ ]:
def json_make_pos(df):
   
    manual_pos = {}

    for ii in df.index:
        
        new_dict = {}
        text = df.loc[ii,"text"]
        tokens = [token.text for sentence in nlp(text).sentences for token in sentence.tokens]
        
        new_dict["aspects"] = []
        for xx in range(1,aspect_number):         
            if df.loc[ii,"aspect_term_"+str(xx)] != None:
                asp_dict = {}   
                term = df.loc[ii,"aspect_term_"+str(xx)]

                # construct aspect position on token level
                asp_toks = [token.text for sentence in nlp(term).sentences for token in sentence.tokens]
                asp_ind = [list(locate(tokens, lambda a: a == tok)) for tok in asp_toks]

                # for aspects appearing only once in text, take the correct position
                # otherwise set to None
                if len(asp_ind[0]) == 1:
                    from_index = asp_ind[0][0]
                else:
                    from_index = None
                if len(asp_ind[-1]) == 1:
                    to_index = asp_ind[-1][0] 
                else: 
                    to_index = None

                # if both start and end pos are unknown, 
                # e.g. for single-word aspects, 
                # take character positions for help
                if from_index == None and to_index == None and len(asp_ind[0]) != 0:
                    print(ii, ": ", text)
                    print("original term: ",term)
                    all_char_from = [i for i in range(len(text)) if text.startswith(asp_toks[0], i)]
                    print("all start chars: ",all_char_from)
                    corr_char_from = int(df.loc[ii, "aspect_from_"+str(xx)])
                    print("correct start char: ",corr_char_from)
                    print("text beginning at correct start char: ", text[corr_char_from:])
                    print("tokens: ",tokens)
                    print("original asp tokens: ", asp_toks)
                    print("original asp indices: ",asp_ind)
                    if corr_char_from == max(all_char_from):
                        from_index = asp_ind[0][-1]
                    elif corr_char_from == min(all_char_from):
                        from_index = asp_ind[0][0]

                # in case of missing start/end positions,
                # try to find "to"/"from" using aspect token number as distance
                if from_index == None and to_index != None:
                    from_index = to_index - len(asp_toks) +1
                if to_index == None and from_index != None:
                    to_index = from_index + len(asp_toks) -1

                # correct tokenization errors in aspect term tokenization
                if from_index == None or to_index == None or asp_toks != tokens[from_index:to_index+1]:

                    print("Tokenization Error in line ",ii,"!")
                    for pos, tok in enumerate(tokens):
                        print(pos, tok)
                    print("aspect term: ", term)
                    print("original asp tokens: ", asp_toks)

                    from_index = int(input("start position?"))
                    to_index = int(input("end position?"))

                    # add manually stated positions to dict for reproducibility
                    manual_pos[text] = {}
                    manual_pos[text][term] = {}
                    manual_pos[text][term]["from"] = from_index
                    manual_pos[text][term]["to"] = to_index
                    print(manual_pos)

                    print("Final aspect tokens: ", tokens[from_index:to_index+1])

                asp_dict["from"] = from_index
                asp_dict["to"] = to_index + 1

    with open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/aspect_positions/semeval_lap_train.json","w") as f:
        json.dump(manual_pos, f)

json_make_pos(data_1)

66 :  It has a 10 hour battery life when you're doing web browsing and word editing, making it perfect for the classroom or office, and in terms of gaming and movie playing it'll have a battery life of just over 5 hours.
original term:  battery life
all start chars:  [17, 180]
correct start char:  17
text beginning at correct start char:  battery life when you're doing web browsing and word editing, making it perfect for the classroom or office, and in terms of gaming and movie playing it'll have a battery life of just over 5 hours.
tokens:  ['It', 'has', 'a', '10', 'hour', 'battery', 'life', 'when', 'you', "'re", 'doing', 'web', 'browsing', 'and', 'word', 'editing', ',', 'making', 'it', 'perfect', 'for', 'the', 'classroom', 'or', 'office', ',', 'and', 'in', 'terms', 'of', 'gaming', 'and', 'movie', 'playing', 'it', "'ll", 'have', 'a', 'battery', 'life', 'of', 'just', 'over', '5', 'hours', '.']
original asp tokens:  ['battery', 'life']
original asp indices:  [[5, 38], [6, 39]]
66 :  It 

## in case of reproducing the dataset

In [ ]:
def json_maker(df, manual_pos):
   
    new_data = []

    for ii in df.index:
        
        new_dict = {}
        text = df.loc[ii,"text"]

        tokens = [token.text for sentence in nlp(text).sentences for token in sentence.tokens]
        new_dict["token"] = tokens

        new_dict["pos"] = [word.xpos for sentence in nlp(text).sentences for word in sentence.words]
        new_dict["head"] = [str(word.head) for sentence in nlp(text).sentences for word in sentence.words]
        new_dict["deprel"] = [word.deprel for sentence in nlp(text).sentences for word in sentence.words]
        
        new_dict["aspects"] = []
        for xx in range(1,aspect_number):         
            if df.loc[ii,"aspect_term_"+str(xx)] != None:
                asp_dict = {}   
                term = df.loc[ii,"aspect_term_"+str(xx)]
                asp_dict["term"] = term
                asp_dict["polarity"] = df.loc[ii,"aspect_polarity_"+str(xx)]

                # construct aspect position on token level
                asp_toks = [token.text for sentence in nlp(term).sentences for token in sentence.tokens]
                asp_ind = [list(locate(tokens, lambda a: a == term)) for term in asp_toks]

                # for aspects appearing only once in text, take the correct position
                # otherwise set to None
                if len(asp_ind[0]) == 1:
                    from_index = asp_ind[0][0]
                else:
                    from_index = None
                if len(asp_ind[-1]) == 1:
                    to_index = asp_ind[-1][0] 
                else: 
                    to_index = None

                # if both start and end pos are unknown, 
                # e.g. for single-word aspects, 
                # take character positions for help
                if from_index == None and to_index == None and len(asp_ind[0]) != 0:

                    all_char_from = [i for i in range(len(text)) if text.startswith(asp_toks[0], i)]
                    corr_char_from = int(df.loc[ii, "aspect_from_"+str(xx)])

                    if corr_char_from == max(all_char_from):
                        from_index = asp_ind[0][-1]
                    elif corr_char_from == min(all_char_from):
                        from_index = asp_ind[0][0]

                # in case of missing start/end positions,
                # try to find "to"/"from" using aspect token number as distance
                if from_index == None and to_index != None:
                    from_index = to_index - len(asp_toks) +1
                if to_index == None and from_index != None:
                    to_index = from_index + len(asp_toks) -1

                # correct tokenization errors in aspect term tokenization
                if from_index == None or to_index == None or asp_toks != tokens[from_index:to_index+1]:
                    if text in manual_pos.keys() and term in manual_pos[text].keys():
                        from_index = manual_pos[text][term]["from"]
                        to_index = manual_pos[text][term]["to"]

                asp_dict["from"] = from_index
                asp_dict["to"] = to_index + 1

                new_dict["aspects"] += [asp_dict]
        
        new_data += [new_dict]

    return new_data

In [ ]:
pos_obj = open("/content/drive/My Drive/Masterarbeit/Data/preprocessing/aspect_positions/semeval_lap_train.json")
loaded_pos = json.load(pos_obj)

for key,val in data_dict.items():

    json_data = json_maker(val, loaded_pos)

    with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/"+key+".json","w") as f:
        json.dump(json_data, f)

# Create LCF-ATEPCdat

In [ ]:
def pol_to_no_shifted(sentiment):
  
    if sentiment == "positive":
        pol = 2
    elif sentiment == "negative":
        pol = 0
    elif sentiment == "neutral":
        pol = 1

    return str(pol)

In [ ]:
def dat_maker(df):
    
    data_lines = []

    for line in df.index:

        text = df.loc[line,"text"]
        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        # create aspect-polarity dict
        asp_sent_dict = {}
        max_asp_len = 0
        for col in range(len(asp_cols)):
            aspect = df.loc[line,asp_cols[col]]
            if aspect != None:
                asp_sent_dict[aspect] = pol_to_no_shifted(df.loc[line,pol_cols[col]])
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())

        label = ""
        # check for one-word-aspects
        for tok in tokens:
            if tok in asp_sent_dict.keys():
                label += tok + " B-ASP -1\n"
            else:
                label += tok + " O -1\n"

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok not in tokens and new_tok in asp_sent_dict.keys():
                    label = label.replace(tokens[no]+" O -1",tokens[no]+" B-ASP -1")
                    for xx in range(1,ii):
                        label = label.replace(tokens[no+xx]+" O -1",tokens[no+xx]+" I-ASP -1")

        # create duplicates of review in case of more than one aspect
        for key, val in asp_sent_dict.items():
            if key in tokens:
                new_label = label.replace(key+" B-ASP -1", key+" B-ASP "+val)
                data_lines += [new_label]
                data_lines += ["\n"]
            else:
                for ii in range(2,max_asp_len+1):
                    for no,tok in enumerate(tokens):
                        new_tok = " ".join(tokens[no:no+ii])
                        if new_tok == key:
                            new_label = label.replace(tokens[no]+" B-ASP -1",tokens[no]+" B-ASP "+val)
                            for xx in range(1,ii):
                                new_label = new_label.replace(tokens[no+xx]+" I-ASP -1",tokens[no+xx]+" I-ASP "+val)

                            data_lines += [new_label]
                            data_lines += ["\n"]

    return data_lines

In [ ]:
for key,val in data_dict.items():

    dat_data = dat_maker(val)

    with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/"+key+".dat","w") as f:
        f.write(''.join(dat_data))

# Create GRACEtxt

In [ ]:
asp_cols = ["aspect_term_"+str(ii) for ii in range(1,aspect_number)]

In [ ]:
import nltk

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('conll2000')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Create chunks and pos tags. Source: https://towardsdatascience.com/chunking-in-nlp-decoded-b4a71b2b4e24


In [ ]:
from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk import ChunkParserI
from nltk.chunk.util import tree2conlltags, conlltags2tree
from nltk.corpus import conll2000

In [ ]:
def conll_tag_chunks(chunk_sents):
    tagged_sents = [tree2conlltags(tree) for tree in chunk_sents]
    return [[(t, c) for (w, t, c) in sent] for sent in tagged_sents]
    
def combined_tagger(train_data, taggers, backoff=None):
    for tagger in taggers:
        backoff = tagger(train_data, backoff=backoff)
    return backoff

class NGramTagChunker(ChunkParserI):

    def __init__(self,train_sentences,tagger_classes=[UnigramTagger,BigramTagger]):
        train_sent_tags=conll_tag_chunks(train_sentences)
        self.chunk_tagger=combined_tagger(train_sent_tags,tagger_classes)
    
    def parse(self,tagged_sentence):
        if not tagged_sentence:
            return None
        pos_tags=[tag for word, tag in tagged_sentence]
        chunk_pos_tags=self.chunk_tagger.tag(pos_tags)
        chunk_tags=[chunk_tag for (pos_tag,chunk_tag) in chunk_pos_tags]
        wpc_tags=[(word,pos_tag,chunk_tag) for ((word,pos_tag),chunk_tag) in zip(tagged_sentence,chunk_tags)]
        return conlltags2tree(wpc_tags)

In [ ]:
data = conll2000.chunked_sents()
ntc = NGramTagChunker(data)

Convert tags into BIOES scheme. Source: https://gist.github.com/allanj/5ad206f7f4645c0269b68fb2065712f4

In [ ]:
def iob_iobes(tags):
    """
    IOB2 (BIO) -> IOBES
    """
    new_tags = []
    for i, tag in enumerate(tags):
        if tag == 'O':
            new_tags.append(tag)
        elif tag.split('-')[0] == 'B':
            if i + 1 != len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('B-', 'S-'))
        elif tag.split('-')[0] == 'I':
            if i + 1 < len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('I-', 'E-'))
        else:
            raise Exception('Invalid IOB format!')
    return new_tags

In [ ]:
def grace_txt_maker(df):
    
    data_lines = []

    for line in df.index:

        text = df.text[line]
        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        # create pos tags
        pos_tags = nltk.pos_tag(tokens)

        # create chunk/phrase tags
        full_tags = tree2conlltags(ntc.parse(pos_tags))
        chunks_list = [full_tags[ii][2] for ii in range(len(full_tags))]
        new_chunks = iob_iobes(chunks_list)

        # create aspect-polarity dict
        asp_sent_dict = {}
        max_asp_len = 0
        for col in range(len(asp_cols)):
            aspect = df.loc[line,asp_cols[col]]
            if aspect != None:
                asp_sent_dict[aspect] = df.loc[line,pol_cols[col]].upper()
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())

        label = ""
        # check for one-word-aspects
        for pos,tok in enumerate(tokens):
            label += tok + " " + pos_tags[pos][1] + " " + new_chunks[pos]
            if tok in asp_sent_dict.keys():
                label +=  " B_AP " + asp_sent_dict[tok] + " B_AP+" + asp_sent_dict[tok] + "\n"
            else:
                label += " O O O \n"

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok not in tokens and new_tok in asp_sent_dict.keys():
                    new_pol = asp_sent_dict[new_tok]
                    label = label.replace(tokens[no]+ " " + pos_tags[no][1] + " " + new_chunks[no] + " O O O \n",
                                          tokens[no]+ " " + pos_tags[no][1] + " " + new_chunks[no] + " B_AP " + \
                                          new_pol + " B_AP+" + new_pol + "\n")
                    for xx in range(1,ii):
                        label = label.replace(tokens[no+xx] + " " + pos_tags[no+xx][1] + " " + new_chunks[no+xx] + " O O O \n",
                                          tokens[no+xx]+ " " + pos_tags[no+xx][1] + " " + new_chunks[no+xx] + " I_AP " + \
                                          new_pol + " I_AP+" + new_pol + "\n")

        data_lines += [label]

    return data_lines

In [ ]:
for key,val in data_dict.items():

    grace_txt_data = grace_txt_maker(val)

    with open("/content/drive/My Drive/Masterarbeit/Data/Final/SEMEVAL-14-LAP/grace_"+key+".txt","w") as f:
        f.write('\n'.join(grace_txt_data))